In [ ]:
#%pip install pyodbc
#%pip install python-dotenv
#%pip install streamlit
#%pip install sqlalchemy
#%pip install langchain
%pip install openai

In [ ]:
# Import necessary libraries and modules
import os
from dotenv import load_dotenv
import streamlit as st
from sqlalchemy import create_engine
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


In [ ]:
# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables using os.getenv
OPENAI_API_TYPE = os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_CHAT_MODEL = os.getenv("OPENAI_CHAT_MODEL")
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")

In [5]:
# Define the ODBC connection string for the SQL Server database
driver = '{ODBC Driver 18 for SQL Server}'
odbc_str = 'mssql+pyodbc:///?odbc_connect=' \
                'Driver='+driver+ \
                ';Server=tcp:' + os.getenv("SQL_SERVER") + \
                ';DATABASE=' + os.getenv("SQL_DB") + \
                ';Uid=' + os.getenv("SQL_USERNAME")+ \
                ';Pwd=' + os.getenv("SQL_PWD") + \
                ';Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=60;'

# Print the ODBC connection string for debugging purposes
print (odbc_str)

mssql+pyodbc:///?odbc_connect=Driver={ODBC Driver 18 for SQL Server};Server=tcp:aqsqlsvr001.database.windows.net;DATABASE=hospitaldb;Uid=arturoqu;Pwd=@DoNotTryThis.1970!;Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=60;


In [6]:
# Create a SQLAlchemy engine object for the SQL Server database
db_engine = create_engine(odbc_str)

# print the engine object for debugging purposes
print (db_engine)

Engine(mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BServer%3Dtcp%3Aaqsqlsvr001.database.windows.net%3BDATABASE%3Dhospitaldb%3BUid%3Darturoqu%3BPwd%3D%40DoNotTryThis.1970%21%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dyes%3BConnection+Timeout%3D60%3B)


In [8]:
# Create an Azure OpenAI chatbot object for natural language processing
llm = AzureChatOpenAI(model=os.getenv("OPENAI_CHAT_MODEL"),
                      deployment_name=os.getenv("OPENAI_CHAT_MODEL"),
                      temperature=0)

/Users/arturoquiroga/autogen-demos/AUTOGEN-STUFF/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/Users/arturoquiroga/autogen-demos/AUTOGEN-STUFF/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://aq-aoaieastus.openai.azure.com/ to https://aq-aoaieastus.openai.azure.com/openai.
  warnings.warn(
/Users/arturoquiroga/autogen-demos/AUTOGEN-STUFF/lib/python3.11/site-packages/langchain_community/chat_models/azure_

In [9]:
# Create a SQL database agent using the Azure OpenAI chatbot and the SQLDatabaseToolkit
db = SQLDatabase(db_engine, schema="SalesLT")
sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sqldb_agent = create_sql_agent(
    llm=llm,
    toolkit=sql_toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
#start a dialog with the chatbot
chat_prompt = ChatPromptTemplate()
chat_prompt.start_dialog(sqldb_agent)



In [10]:
# Define Streamlit app
def app():
    # Set the title and logo of the Streamlit app
    st.title("Arturo's AdventureWorks2023 SQL Query Tool")
    st.image("https://biendata.xyz/media/7c92a358-84ff-4d88-b44b-33861780b703_logo.png", use_column_width=True)
    
    # Prompt the user to enter a question and provide a text input field
    st.write("Enter your question below and click 'Submit' to get an answer.")
    question = st.text_input("Question:")
    
    # When the user clicks the "Submit" button, generate a response using the SQL database agent
    if st.button("Submit"):
        final_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", 
                 """
                  You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about Categories, Products and Orders.
                 """
                ),
                ("user", f"{question}\n ai: "),
            ]
        )
        response = sqldb_agent.run(final_prompt)
        
        # Display the response in the Streamlit app's output area
        st.write("Output:")
        st.write(response)

In [11]:
# Run the Streamlit app
if __name__ == "__main__":
    app()


2024-05-21 12:46:07.177 
  command:

    streamlit run /Users/arturoquiroga/autogen-demos/AUTOGEN-STUFF/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-05-21 12:46:07.192 Session state does not function when running a script without `streamlit run`
